In [1]:
# Add analysis to the path
import sys
import pathlib
import os
import numpy
import pandas

parent_dir = os.path.join(os.path.abspath(''), "..")
sys.path.append(parent_dir)

import parsers
from pprint import pprint

path_to_results = os.path.normpath(os.path.join(parent_dir, "archive"))
print(path_to_results)
working_dir = os.path.normpath(os.path.join(parent_dir, "working"))
data = parsers.main(path_to_results, working_dir=working_dir)
print("Finished")

/Users/bhanugarg/Documents/school/cs4365/analysis/archive
Extracting 10 top level archives on 4 workers
Finished


In [2]:
print(data.get("ii-mc-b").replicas[0].config())

{'USERNAME': 'jazevedo', 'HOSTS_TYPE': 'physical', 'HARDWARE_TYPE': 'c8220', 'ENABLED_CPUS': 4, 'USE_PATCHED_DOCKER': 1, 'ENABLE_RADVISOR': 0, 'ENABLE_COLLECTL': 1, 'POLLING_INTERVAL': '6s', 'COLLECTION_INTERVAL': 50, 'WEB_HOSTS': 'clnode048.clemson.cloudlab.us', 'POSTGRESQL_HOST': 'clnode054.clemson.cloudlab.us', 'WORKER_HOSTS': 'clnode072.clemson.cloudlab.us', 'MICROBLOG_HOSTS': 'clnode080.clemson.cloudlab.us', 'MICROBLOG_PORT': 9090, 'AUTH_HOSTS': 'clnode062.clemson.cloudlab.us', 'AUTH_PORT': 9091, 'INBOX_HOSTS': 'clnode053.clemson.cloudlab.us', 'INBOX_PORT': 9092, 'QUEUE_HOSTS': 'clnode094.clemson.cloudlab.us', 'QUEUE_PORT': 9093, 'SUB_HOSTS': 'clnode055.clemson.cloudlab.us', 'SUB_PORT': 9094, 'CLIENT_HOSTS': 'clnode095.clemson.cloudlab.us', 'WORKLOAD_CONFIG': 'conf/bursty.yml', 'SESSION_CONFIG': 'conf/session.yml', 'APACHE_PROCESSES': 8, 'APACHE_THREADSPERPROCESS': 4, 'POSTGRES_MAXCONNECTIONS': 175, 'NUM_WORKERS': 32, 'MICROBLOG_THREADPOOLSIZE': 32, 'AUTH_THREADPOOLSIZE': 32, 'INB

In [27]:
import csv
import numpy
import sys
import pandas as pd

class LogEntry:

    def __init__(self, event, ts, sock_fd):
        self._event = event
        self._ts = ts
        self._sock_fd = sock_fd

    def __lt__(self, other):
        return self._ts < other._ts

    def event(self):
        return self._event

    def ts(self):
        return self._ts

    def sock_fd(self):
        return self._sock_fd

    def __repr__(self):
        return "[{event} -- TS: {ts}; SOCK_FD: {sock_fd}]".format(event=self._event, ts=str(self._ts), sock_fd=str(self._sock_fd))

    

def aggregate(test_name, rep_num):
    recvfrom = data.get(test_name).replicas[rep_num].hosts["client"].recvfrom()
    sendto = data.get(test_name).replicas[rep_num].hosts["client"].sendto()
    connect = data.get(test_name).replicas[rep_num].hosts["client"].connect()

    log_entries = []
    for i in range(len(connect)):
        if int(connect[i].port()) == 80:
            log_entries.append(LogEntry("connect", int(connect[i].ts()), int(connect[i].sock_fd())))

    for i in range(len(sendto)):     
        log_entries.append(LogEntry("sendto", int(sendto[i].ts()), int(sendto[i].sock_fd())))

    for i in range(len(recvfrom)):   
        log_entries.append(LogEntry("recvfrom", int(recvfrom[i].ts()), int(recvfrom[i].sock_fd())))

    log_entries.sort()
    requests = []
    for i in range(len(log_entries)):
        if log_entries[i].event() == "connect":
            request = [log_entries[i]]
            j = i + 1
            while j < len(log_entries) and(log_entries[j].event() != "connect"or log_entries[i].sock_fd() != log_entries[j].sock_fd()):
                if log_entries[i].sock_fd() == log_entries[j].sock_fd():
                    request.append(log_entries[j])
                j += 1
            requests.append(request)

    rt_dist = []
    for request in requests:
        if request[-1].ts() - request[0].ts() > 3000000:
            dif = (request[-1].ts() - request[0].ts())/(1000000)
            rt_dist.append((request[0].ts(), dif))

    df = pd.DataFrame({test_name:rt_dist})
    return df

In [41]:
dic = {}
output = os.listdir('/Users/bhanugarg/Documents/school/cs4365/analysis/working')

for i in output:
    temp = i.split(".",1)[0]
    df = pd.DataFrame()
    if temp is "ii-mc-b":
        for j in range(18):
            df.append(aggregate(temp, j))
    else:   
        for j in range(20):
            df.append(aggregate(temp, j))
    dic[i] = df
print(dic)

IndexError: list index out of range

In [43]:
print(aggregate('ii-c-s', 19))

                           ii-c-s
0    (1587683646067663, 14.67223)
1     (1587683648344794, 3.80581)
2    (1587683675751308, 9.559722)
3    (1587683686296397, 14.40719)
4    (1587683706336259, 3.779441)
5    (1587683710115476, 4.509164)
6    (1587683714611931, 8.638876)
7   (1587683714624413, 65.690654)
8    (1587683719462133, 3.784821)
9    (1587683725008265, 3.478973)
10   (1587683733350097, 3.328811)
11   (1587683733363152, 4.164124)
12   (1587683737957273, 6.767495)
13   (1587683745180466, 5.690681)
14   (1587683745180742, 5.663974)
15  (1587683750871214, 29.431777)
16   (1587683757361763, 4.497252)
17   (1587683757361827, 5.809438)
18   (1587683763171332, 9.093721)
19   (1587683765439309, 4.766932)
20   (1587683773134324, 7.160767)
21   (1587683780303061, 18.20839)
22  (1587683780315121, 18.204136)
23   (1587683780775347, 5.879156)
24   (1587683790747171, 5.031697)
25   (1587683798506127, 3.269085)
26   (1587683798511503, 3.267978)
27  (1587683802123278, 82.497723)
28   (15876838